# Data Ingestion Pipeline

- Import important libraries
- Move data from cached kagglehub dataset folder into data/raw folder in project


In [19]:
import kagglehub
import shutil
from pathlib import Path

In [6]:
path = kagglehub.dataset_download("bryanb/fifa-player-stats-database")

100%|██████████| 13.3M/13.3M [00:01<00:00, 7.26MB/s]

Extracting files...


In [7]:
print("Path to dataset files:", path)

Path to dataset files: C:\Users\HP\.cache\kagglehub\datasets\bryanb\fifa-player-stats-database\versions\35


In [20]:
if Path("../data/raw/35"):
    print("data exists!")
else:    
    shutil.move(f"C:/Users/HP/.cache/kagglehub/datasets/bryanb/fifa-player-stats-database/versions/35","../data/raw")

data exists!
